# Credit NOTE

This notebook is just Thai comments on my RSNA teammate top kernel, with minor modification to make it (1) simpler, (2) works on CPU and (3) work on 2nd stage data : https://www.kaggle.com/akensert/inceptionv3-prev-resnet50-keras-baseline-model 

# เกริ่นนำ

สวัสดีครับเพื่อนๆ หลังจากที่เราได้เล่าปัญหาเรื่องการตรวจสอบเลือดคั่งในสมอง 5 รูปแบบ มุมมองต่อปัญหาทั้งเชิงวิศวกร และเชิงรังสีแพทย์ไปแล้ว (ดูย้อนหลังได้ที่นี่ี http://bit.ly/thaikeras-rsna ) ในโน้ตบุ้คนี้เราจะมาดู โค้ด Keras ที่นำความรู้จากบทความที่ผ่านมา มาสร้างโมเดลต้นแบบกันครับ สำหรับเพื่อนๆ ที่ไม่เคยใช้งาน Kaggle มาก่อนสามารถดูขั้นตอนการเริ่มต้นได้ง่ายๆ ที่นี่ครับ : https://thaikeras.com/2018/setup-kaggle-workshop/

โน้ตบุ้คฉบับนี้เป็นของเพื่อนชาวสวีเดน (@akensert) ที่ร่วมทีมกับทีม ThaiKeras ในการแข่งขัน RSNA ที่ผ่านมา ซึ่งเป็นโน้ตบุ้คที่สร้าง baseline ความแม่นยำสูงสุด อ่านง่ายและเป็นระเบียบ และได้รับการชื่นชมจากเพื่อนๆ ใน Kaggle community เป็นอย่างมากครับ

โดยเนื้อหาในโค้ด จะแบ่งออกเป็น 5 หัวข้อดังต่อไปนี้ครับ (เพื่อนๆ ที่เปิดดูด้วย PC จะสามารถเลือกไปหัวข้อที่ต้องการได้จากเมนูบน Kaggle ด้านซ้ายครับ)

**1. Preprocessing / Windowing -- ** โค้ดในการทำ windowing จากภาพ Dicom ดั่งที่อธิบายอย่างละเอียดในบทความฉบับก่อน

**2. Data Generator -- ** โค้ดในการจัดการข้อมูลทีละ batch เนื่องจากในปัญหาที่ข้อมูลมีจำนวนมหาศาลเช่นนี้ั เราไม่สามารถโหลดข้อมูลขึ้นมาพร้อมๆ กันได้ เราจึงจำเป็นต้องสร้าง Keras Data Generator ขึ้นมาเพื่อจัดการกับข้อมูลทีละ Batch ในระหว่างฝึกสอนโมเดล

**3. Model, Loss and Metric -- ** โค้ดสำหรับสร้างโมเดล รวมทั้ง objective function ในงานนี้

**4. Meta-data (CSV) -- ** โค้ดสำหรับ import และจัดการข้อมูล meta data ที่สำคัญจาก CSV มายู่ใน panda และ numpy

**5. Train and Predict -- ** ใช้โค้ดทั้งหมดเพื่อเริ่มฝึกสอนและทำนายข้อมูล test data

หมายเหตุ เมื่อ Copy notebook เพื่อทดลองโค้ด เพื่อนๆ ต้องเปิด GPU ในเมดูของ Kaggle Notebook ที่อยู่ด้านขวามือนะครับ (จำกัดการใช้งานฟรี 30 ชั่วโมงต่อ 1 สัปดาห์ โดยโน้ตบุ้คนี้รันและเมื่อเซ็ตค่า GPU = On และ EPOCHS = 5 จะใช้เวลารันราวๆ 7-8 ชั่วโมงครับ

In [ ]:
EPOCHS = 0 # แก้จำนวน Epochs ที่จะเทรน โดยค่าสูงสุดที่สามารถรันได้ใน Kaggle Notebook (จำกัดชั่วโมงในการรันไม่เกิน 9 ชั่วโมง) คือ EPOCHS = 5

In [ ]:
import numpy as np
import pandas as pd
import pydicom
import os
import matplotlib.pyplot as plt
import collections
from tqdm import tqdm_notebook as tqdm
from datetime import datetime

from math import ceil, floor, log
import cv2

import tensorflow as tf
import keras

import sys

# from keras_applications.resnet import ResNet50
from keras_applications.inception_v3 import InceptionV3

from sklearn.model_selection import ShuffleSplit



ก่อนอื่นกำหนด path ของ train และ test data ครับ

In [ ]:
!ls ../input/rsna-intracranial-hemorrhage-detection/rsna-intracranial-hemorrhage-detection

In [ ]:
test_images_dir = '../input/rsna-intracranial-hemorrhage-detection/rsna-intracranial-hemorrhage-detection/stage_2_test/'
train_images_dir = '../input/rsna-intracranial-hemorrhage-detection/rsna-intracranial-hemorrhage-detection/stage_2_train/'

In [ ]:
!ls {train_images_dir} | wc # นับจำนวนรูปใน training data

# 1. Preprocessing (brain + subudral + soft tissue)
Idea Credits: [Ryan Epp](https://www.kaggle.com/reppic/gradient-sigmoid-windowing), [David Tang](https://www.kaggle.com/dcstang/see-like-a-radiologist-with-systematic-windowing), [Marco](https://www.kaggle.com/marcovasquez/basic-eda-data-visualization), [Nanashi](https://www.kaggle.com/jesucristo/rsna-introduction-eda-models), [Richard McKinley](https://www.kaggle.com/omission/eda-view-dicom-images-with-correct-windowing)), [appian42's repo](https://github.com/appian42/kaggle-rsna-intracranial-hemorrhage/) (windowing), [Jeremy's kernel](https://www.kaggle.com/jhoward/cleaning-the-data-for-rapid-prototyping-fastai) (cleaning)

ในโค้ดข้างล่างเรานิยามฟังก์ชัน `window_image` เพื่อที่จะดึงข้อมูลเฉพาะ window ที่ต้องการ ส่วนฟังก์ชัน `bsb_window` จะทำการเรียก `window_image` 3 ครั้ง เพื่อดึงข้อมูลใน brain window, subdural window และ soft-tissue window ตามลำดับ และ normalize ข้อมูลให้อยู่ในช่วง [0,255] เช่นเดียวกับ format ของรูป 3-channel ปกติ

In [ ]:
def window_image(dcm, window_center, window_width):
    
    img = dcm.pixel_array * dcm.RescaleSlope + dcm.RescaleIntercept
    img_min = window_center - window_width // 2
    img_max = window_center + window_width // 2
    img = np.clip(img, img_min, img_max)

    return img

def bsb_window(dcm):
    brain_img = window_image(dcm, 40, 80)
    subdural_img = window_image(dcm, 80, 200)
    soft_img = window_image(dcm, 40, 380)
    
    brain_img = (brain_img - 0) / 80
    subdural_img = (subdural_img - (-20)) / 200
    soft_img = (soft_img - (-150)) / 380 # soft-tissue window
    bsb_img = np.array([brain_img, subdural_img, soft_img]).transpose(1,2,0)

    return bsb_img

ซึ่งเมื่อเราเรียกฟังก์ชัน `bsb_window` ด้วยไฟล์ dicom เราก็จะได้ 3-dim (กว้าง, ยาว, 3) numpy array กลับมาครับ

ฟังก์ชัน `_read` นั้นได้ automate กระบวนการที่จำเป็นทั้งหมดไว้ครับโดย 

* อ่าน dicom จากชื่อ path ของไฟล์ที่ต้องการ
* แปลง dicom เป็นรูปภาพโดยเรียก `bsb_window` และในกรณีที่ไม่มีรูปภาพที่ต้องการก็จะส่ง zero numpy array กลับมาแทน
* เปลี่ยนขนาดของรูปภาพให้เล็กลงเพื่อให้กระบวนการฝึกสอนโมเดลเร็วขึ้น (ค่า default คือ 256x256 กว้างและยาวตามลำดับ)

In [ ]:
def _read(path, desired_size=(256, 256)):
    """Will be used in DataGenerator"""
    
    dcm = pydicom.dcmread(path)
    
    try:
        img = bsb_window(dcm)
    except:
        img = np.zeros(desired_size)
    
    
    img = cv2.resize(img, desired_size[:2], interpolation=cv2.INTER_LINEAR)
    
    return img

# ทดสอบโชว์รูปใน 3 windows
# ลองเปลี่ยนชื่อไฟล์เหล่านี้ดู: ID_2669954a7, ID_5c8b5d701, ID_52c9913b1

plt.imshow(
    _read(train_images_dir+'ID_5c8b5d701'+'.dcm', (256, 256)), cmap=plt.cm.bone
);

# ในรูปนี้มีปรากฏการณ์เลือดคั่งในสมอง 3 รูปแบบด้วยกัน เพื่อนๆ มองออกไหมครับ ?
#                                     ID  Label
# 4045566          ID_5c8b5d701_epidural      0
# 4045567  ID_5c8b5d701_intraparenchymal      1
# 4045568  ID_5c8b5d701_intraventricular      0
# 4045569      ID_5c8b5d701_subarachnoid      1
# 4045570          ID_5c8b5d701_subdural      1
# 4045571               ID_5c8b5d701_any      1

# 2. Data generators

สร้าง DataGenerator Object จาก class `keras.utils.Sequence` ซึ่งเรื่อง DataGenerator ที่มีประโยชน์มากๆ ในการใช้งานจริงเวลาเรามีข้อมูลปริมาณมหาศาลในมือนี้ ได้ใช้หลายรอบใน workshop ของ ThaiKeras ที่ผ่านมา เดี๋ยวเราจะมีบทความ Series ที่อธิบายเรื่อง Keras อย่างละเอียดในปีนี้ ขอเพื่อนๆ อดใจรอครับ :D

ณ ตอนนี้ เพื่อนๆ ทำความเข้าใจเบื้องต้นก่อนว่าการสร้าง DataGenerator จะเป็นการบอกให้ Keras โหลดและจัดการข้อมูลได้ทีละ batch แทนที่จะต้องโหลดทีเดียวทั้ง Dataset ซึ่งไม่สามารถทำได้ใน RAM ที่มีน้อยนิดใน machine ทั่วๆ ไปครับ


In [ ]:
class DataGenerator(keras.utils.Sequence):

    def __init__(self, list_IDs, labels=None, batch_size=1, img_size=(512, 512, 1), 
                 img_dir=train_images_dir, *args, **kwargs):

        self.list_IDs = list_IDs
        self.labels = labels
        self.batch_size = batch_size
        self.img_size = img_size
        self.img_dir = img_dir
        self.on_epoch_end()

    def __len__(self):
        return int(ceil(len(self.indices) / self.batch_size))

    def __getitem__(self, index):
        indices = self.indices[index*self.batch_size:(index+1)*self.batch_size]
        list_IDs_temp = [self.list_IDs[k] for k in indices]
        
        if self.labels is not None:
            X, Y = self.__data_generation(list_IDs_temp)
            return X, Y
        else:
            X = self.__data_generation(list_IDs_temp)
            return X
        
    def on_epoch_end(self):
        
        
        if self.labels is not None: # for training phase we undersample and shuffle
            # keep probability of any=0 and any=1
            keep_prob = self.labels.iloc[:, 0].map({0: 0.35, 1: 0.5})
            keep = (keep_prob > np.random.rand(len(keep_prob)))
            self.indices = np.arange(len(self.list_IDs))[keep]
            np.random.shuffle(self.indices)
        else:
            self.indices = np.arange(len(self.list_IDs))

    def __data_generation(self, list_IDs_temp):
        X = np.empty((self.batch_size, *self.img_size))
        
        if self.labels is not None: # training phase
            Y = np.empty((self.batch_size, 6), dtype=np.float32)
        
            for i, ID in enumerate(list_IDs_temp):
                X[i,] = _read(self.img_dir+ID+".dcm", self.img_size)
                Y[i,] = self.labels.loc[ID].values
        
            return X, Y
        
        else: # test phase
            for i, ID in enumerate(list_IDs_temp):
                X[i,] = _read(self.img_dir+ID+".dcm", self.img_size)
            
            return X

# 3. Metric and Model

ในหัวข้อนี้เป็นตัวอย่างการกำหนด loss/metric (3a.) และโมเดล (3b.) โดย loss และ metric นั้นถูกกำหนดจากสมาคม RSNA ที่เป็นจัดเตรียมข้อมูล และกำหนดกติกาการแข่งขัน

### 3a. loss function and metric

In [ ]:
from keras import backend as K

def weighted_log_loss(y_true, y_pred):
    """
    Can be used as the loss function in model.compile()
    ---------------------------------------------------
    """
    
    class_weights = np.array([2., 1., 1., 1., 1., 1.])
    
    eps = K.epsilon()
    
    y_pred = K.clip(y_pred, eps, 1.0-eps)

    out = -(         y_true  * K.log(      y_pred) * class_weights
            + (1.0 - y_true) * K.log(1.0 - y_pred) * class_weights)
    
    return K.mean(out, axis=-1)


def _normalized_weighted_average(arr, weights=None):
    """
    A simple Keras implementation that mimics that of 
    numpy.average(), specifically for this competition
    """
    
    if weights is not None:
        scl = K.sum(weights)
        weights = K.expand_dims(weights, axis=1)
        return K.sum(K.dot(arr, weights), axis=1) / scl
    return K.mean(arr, axis=1)


def weighted_loss(y_true, y_pred):
    """
    Will be used as the metric in model.compile()
    ---------------------------------------------
    
    Similar to the custom loss function 'weighted_log_loss()' above
    but with normalized weights, which should be very similar 
    to the official competition metric:
        https://www.kaggle.com/kambarakun/lb-probe-weights-n-of-positives-scoring
    and hence:
        sklearn.metrics.log_loss with sample weights
    """
    
    class_weights = K.variable([2., 1., 1., 1., 1., 1.])
    
    eps = K.epsilon()
    
    y_pred = K.clip(y_pred, eps, 1.0-eps)

    loss = -(        y_true  * K.log(      y_pred)
            + (1.0 - y_true) * K.log(1.0 - y_pred))
    
    loss_samples = _normalized_weighted_average(loss, class_weights)
    
    return K.mean(loss_samples)


def weighted_log_loss_metric(trues, preds):
    """
    Will be used to calculate the log loss 
    of the validation set in PredictionCheckpoint()
    ------------------------------------------
    """
    class_weights = [2., 1., 1., 1., 1., 1.]
    
    epsilon = 1e-7
    
    preds = np.clip(preds, epsilon, 1-epsilon)
    loss = trues * np.log(preds) + (1 - trues) * np.log(1 - preds)
    loss_samples = np.average(loss, axis=1, weights=class_weights)

    return - loss_samples.mean()



### 3b. Model

ในการสร้างโมเดล Keras เพื่อเรียนรู้และทำนายข้อมูลนี้ จะคล้ายกับ Workshops อื่นๆ ที่ผ่านมาครับ (https://thaikeras.com/2018/setup-kaggle-workshop/) 
โดยในอนาคต (คาดว่าภายในปี 2020) เราจะมี Series บทความที่สอนการสร้างโมเดล Keras อย่างละเอียด step-by-step ทยอยออกมาครับ

สำหรับภาพรวมคือเราสร้างโมเดล image classifier มาตรฐานที่รับ input เป็นรูป (ซึ่งจัดการโดย DataGenerator) และ Output เป็นความน่าจะเป็นที่จะมีเลือดคั่ง 6 ค่าตามที่อธิบายไว้ใน[บทความก่อน](http://bit.ly/thaikeras-rsna)ครับ 

In [ ]:

class PredictionCheckpoint(keras.callbacks.Callback):
    
    def __init__(self, test_df, valid_df, 
                 test_images_dir=test_images_dir, 
                 valid_images_dir=train_images_dir, 
                 batch_size=32, input_size=(224, 224, 3)):
        
        self.test_df = test_df
        self.valid_df = valid_df
        self.test_images_dir = test_images_dir
        self.valid_images_dir = valid_images_dir
        self.batch_size = batch_size
        self.input_size = input_size
        
    def on_train_begin(self, logs={}):
        self.test_predictions = []
        self.valid_predictions = []
        
    def on_epoch_end(self,batch, logs={}):
        print('ทำนาย test data...')
        self.test_predictions.append(
            self.model.predict_generator(
                DataGenerator(self.test_df.index, None, self.batch_size, self.input_size, self.test_images_dir), verbose=1)[:len(self.test_df)])

class MyDeepModel:
    
    def __init__(self, engine, input_dims, batch_size=5, num_epochs=4, learning_rate=1e-3, 
                 decay_rate=1.0, decay_steps=1, weights="imagenet", verbose=1):
        
        self.engine = engine
        self.input_dims = input_dims
        self.batch_size = batch_size
        self.num_epochs = num_epochs
        self.learning_rate = learning_rate
        self.decay_rate = decay_rate
        self.decay_steps = decay_steps
        self.weights = weights
        self.verbose = verbose
        self._build()

    def _build(self):
        
        
        engine = self.engine(include_top=False, weights=self.weights, input_shape=self.input_dims,
                             backend = keras.backend, layers = keras.layers,
                             models = keras.models, utils = keras.utils)
        
        x = keras.layers.GlobalAveragePooling2D(name='avg_pool')(engine.output)
        out = keras.layers.Dense(6, activation="sigmoid", name='dense_output')(x)

        self.model = keras.models.Model(inputs=engine.input, outputs=out)

        self.model.compile(loss="binary_crossentropy", optimizer=keras.optimizers.Adam(), metrics=[weighted_loss])
    

    def fit_and_predict(self, train_df, valid_df, test_df):
        
        # callbacks
        pred_history = PredictionCheckpoint(test_df, valid_df, input_size=self.input_dims)
        scheduler = keras.callbacks.LearningRateScheduler(lambda epoch: self.learning_rate * pow(self.decay_rate, floor(epoch / self.decay_steps)))
        
        self.model.fit_generator(
            DataGenerator(
                train_df.index, 
                train_df, 
                self.batch_size, 
                self.input_dims, 
                train_images_dir
            ),
            epochs=self.num_epochs,
            verbose=self.verbose,
            use_multiprocessing=True,
            workers=4,
            callbacks=[pred_history, scheduler]
        )
        
        return pred_history
    
    def save(self, path):
        self.model.save_weights(path)
    
    def load(self, path):
        self.model.load_weights(path)

# 4. Read csv files

อ่านไฟล์ข้อมูล meta-data ของ DICOM ที่สมาคม RSNA และ Kaggle จัดเตรียมไว้ให้ ซึ่งเราจะคัดเฉพาะข้อมูลที่เราต้องการ ซึ่งในที่้คือ ชื่อรูป และ label ว่ามีเลือดคั่งในสมองรูปแบบไหนบ้างเท่านั้น โดยข้อมูลอื่นเช่น ประวัติคนไข้ ต่างๆ จะยังไม่ได้เอามาใช้ ในโมเดลพื้นฐานนี้ครับ

หลังจากนั้นก็จะจัดเรียงให้อยู่ในรูปตารางเพื่อเก็บใน pandas dataframe เพื่อความง่ายในการใช้งาน

In [ ]:
def read_testset(filename="../input/rsna-intracranial-hemorrhage-detection/rsna-intracranial-hemorrhage-detection/stage_2_sample_submission.csv"):
    df = pd.read_csv(filename)
    df["Image"] = df["ID"].str.slice(stop=12)
    df["Diagnosis"] = df["ID"].str.slice(start=13)
    
    df = df.loc[:, ["Label", "Diagnosis", "Image"]]
    df = df.set_index(['Image', 'Diagnosis']).unstack(level=-1)
    
    return df

def read_trainset(filename="../input/rsna-intracranial-hemorrhage-detection/rsna-intracranial-hemorrhage-detection/stage_2_train.csv"):
    df = pd.read_csv(filename)
    df["Image"] = df["ID"].str.slice(stop=12)
    df["Diagnosis"] = df["ID"].str.slice(start=13)
    
    # เอารูปภาพซ้ำออก
    duplicates_to_remove = [56340, 56341, 56342, 56343, 56344, 56345, 56346, 56347, 56348, 56349, 56350, 56351, 
                            1171824, 1171825, 1171826, 1171827, 1171828, 1171829, 1171830, 1171831, 1171832, 1171833, 
                            1171834, 1171835, 3705306, 3705307, 3705308, 3705309, 3705310, 3705311, 3705312, 3705313, 
                            3705314, 3705315, 3705316, 3705317, 3842472, 3842473, 3842474, 3842475, 3842476, 3842477, 
                            3842478, 3842479, 3842480, 3842481, 3842482, 3842483]
    
    df = df.drop(index=duplicates_to_remove)
    df = df.reset_index(drop=True)
    
    df = df.loc[:, ["Label", "Diagnosis", "Image"]]
    df = df.set_index(['Image', 'Diagnosis']).unstack(level=-1)
    
    return df

df = read_trainset()
test_df = read_testset()

In [ ]:
# ตัวอย่าง meta-data ของ training set ที่เราต้องการ
df.head(3)

In [ ]:
# ตัวอย่าง meta-data ของ test set ซึ่งเราจะเปลี่ยนค่า 0.5 ในตาราง ให้เป็นคำนายเลือดคั่งในสมองจากโมเดลของเราภายหลัง
test_df.head(3)

# 5. Train model and predict

Keras นั้นออกแบบให้การฝึกสอนโมเดลสามารถทำได้ภายในบรรทัดเดียวครับ ในที่นี้เราใช้ฟังก์ชัน `fit_and_predict` ที่เราได้เขียนไว้ในหัวข้อ 4 เพื่อเรียนรู้จาก training data และทำนาย test data

โดยเราจะแบ่ง training data ออกเป็นสองกลุ่มคือกลุ่มฝึกสอนจริงๆ (`training_idx`) และกลุ่มที่ใช้ตรวจสอบความแม่นยำ (`valid_idx`) ด้วยฟังก์ชัน `ShuffleSplit` ซึ่งเป็นการแบ่งกลุ่มแบบสุ่มครับ (ดูรายละเอียด https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.ShuffleSplit.html)

In [ ]:
# train set (00%) and validation set (10%)
ss = ShuffleSplit(n_splits=10, test_size=0.1, random_state=42).split(df.index)

# lets go for the first fold only
train_idx, valid_idx = next(ss)
# EPOCHS=0
if EPOCHS == 0: # just for illustration on CPU
    train_idx = train_idx[:100]
    valid_idx = valid_idx[:100]
    EPOCHS = 1

# obtain model
model = MyDeepModel(engine=InceptionV3, input_dims=(256, 256, 3), batch_size=32, learning_rate=5e-4,
                    num_epochs=EPOCHS, decay_rate=0.8, decay_steps=1, weights="imagenet", verbose=1)

# obtain test + validation predictions (history.test_predictions, history.valid_predictions)
history = model.fit_and_predict(df.iloc[train_idx], df.iloc[valid_idx], test_df)


สุดท้ายเราจะนำคำนายของ test data เพื่อทำให้อยู่ในรูป dataframe ที่ผู้จัดการแข่งขันกำหนดครับ

In [ ]:
test_df = read_testset()

print(history.test_predictions)

test_df.iloc[:, :] = np.mean(history.test_predictions, axis=0)

test_df = test_df.stack().reset_index()

test_df.insert(loc=0, column='ID', value=test_df['Image'].astype(str) + "_" + test_df['Diagnosis'])

test_df = test_df.drop(["Image", "Diagnosis"], axis=1)

test_df.to_csv('submission.csv', index=False)
test_df.head()

โมเดลนี้เป็นเพียงโมเดลพื้นฐานเท่านั้น และยังสามารถปรับปรุงให้แม่นยำได้อีกหลายทิศทางครับ โดยเพื่อนๆ สามารถดูไอเดียเพิ่มเติมอีกมากมายได้ที่นี้ครับ https://www.kaggle.com/c/rsna-intracranial-hemorrhage-detection/discussion